In [1]:
# THERE ARE SEPARATE NOTEBOOKS FOR VISUALIZATIONS, DATASET ANALYSIS, ETC. IN THE REPO.

import pandas as pd
import numpy as np

# READ THE CSV INTO DATAFRAME

df = pd.read_csv('Syngenta/Syngenta_2017/Experiment_dataset.csv')

In [ ]:
# THIS IS USED WHEN CONNECTING TO COMPUTE NODES TO CHECK IF EVERYTHING IS WORKING
import os
print(os.getcwd())

In [ ]:
# CURRENTLY NECESSARY IF: USING 174 ADDITIONAL VARIETY COLUMNS METHOD

variety_dummies = pd.get_dummies(df.Variety)
df = pd.concat([df, variety_dummies], axis=1)

In [ ]:
# CURRENTLY NECESSARY IF: USING 8 ADDITIONAL VARIETY COLUMNS (BINARY REPRESENTATION)

UNIQUE_VARIETIES = np.unique(df.Variety)
INDICES = [i for i in range(0, len(UNIQUE_VARIETIES))]
print(np.where(UNIQUE_VARIETIES=='V000017'))
df.Variety = df.Variety.apply(lambda var: np.where(UNIQUE_VARIETIES==var)[0][0])
print(UNIQUE_VARIETIES)
print(df.Variety)

df.Variety = df.Variety.apply(lambda var: '{0:08b}'.format(var))
print("done")
df.Variety = df.Variety.apply(lambda var: list(var))
print("done")
variety_binary_df = pd.DataFrame(pd.Series(df.Variety[i]) for i in range(0, len(df.Variety)))
df = pd.concat([df, variety_binary_df], axis=1)

In [3]:
# GOAL OF THIS MODULE:
# Encode the planting date as a season
# NEW GOAL:
# GET DUMMIES FOR SEASONS

# remove the dates that are "."
df = df[~df['Planting date'].str.match("\.")]
plant_date = df['Planting date'].apply(lambda dt: pd.to_datetime(dt))
plant_months = plant_date.apply(lambda dt: dt.month)
season = plant_date.rename("Season")
season = pd.to_datetime(season)
season = season.apply(lambda dt: (dt.month%12 + 3)//3)
# df['Plant date'] = pd.to_datetime(df['Plant date'])
df = pd.concat([df, season], axis=1)

# plant_date = pd.to_datetime(df['Planting date'], infer_datetime_format=True)
# df = df['Planting date'].apply(lambda dt: (dt.month%12 + 3)//3)
# pd.get_dummies(df['Planting date'])


In [4]:
# ADD MONTH OF MAY AND JUNE ONE HOT ENCODING INTO THE DATAFRAME
pd.get_dummies(plant_months).sum()
june = pd.get_dummies(plant_months).loc[:,6]
june = june.rename("June")
may = pd.get_dummies(plant_months).loc[:,5]
may = may.rename("May")
df = pd.concat([df, may], axis=1)
df = pd.concat([df, june], axis=1)

In [5]:
# LATITUDE AND LONGITUDE CLUSTERING INTO FEATURES

from sklearn.cluster import KMeans

latlong = df.loc[:, ['Latitude', 'Longitude']]

kmeans = KMeans(n_clusters=4, random_state=0).fit(latlong)
kmeans.labels_.shape
lat_long_dummies = pd.get_dummies(kmeans.labels_)
lat_long_dummies = lat_long_dummies.rename(index=int, columns={0: "Loc Clust 0",
                                                               1: "Loc Clust 1",
                                                               2: "Loc Clust 2",
                                                               3: "Loc Clust 3"})
df = pd.concat([df, lat_long_dummies], axis = 1)

In [6]:
#REMOVE ANY NAN VALUES

print(df.columns)
df = df[~df.Silt.isnull()]
df = df[~df['Loc Clust 1'].isnull()]

Index(['Experiment', 'Location', 'Variety', 'Planting date', 'Yield',
       'Check Yield', 'Yield difference', 'Year', 'Latitude', 'Longitude',
       'Temperature', 'Precipitation', 'Solar Radiation', 'Soil class', 'CEC',
       'Organic matter', 'pH', 'Clay', 'Silt', 'Sand', 'PI', 'Area', 'Season',
       'May', 'June', 'Loc Clust 0', 'Loc Clust 1', 'Loc Clust 2',
       'Loc Clust 3'],
      dtype='object')


In [ ]:
for col in df.columns:
    print(col, type(df[col][0]))

In [7]:
# DROP ALL THE CELLS THAT ARE NOT USABLE SUCH AS THE ONES THAT ARE STRINGS OR DATES

# set if want to drop some columns specifically
should_drop = 1
columns_to_drop = ['Experiment', 'Location',
                   'Check Yield', 'Yield difference', 'Latitude',
                   'Longitude', 'PI', 'Variety', 'Planting date', 'Season']

# set if want to keep some columns specifically
should_keep = 0
# columns_to_keep = ['Loc Clust 0', 'Loc Clust 1', 'Loc Clust 2', 'Loc Clust 3']
columns_to_keep_top = ['Silt', 'Precipitation', 'Temperature', 'Solar Radiation', 'Organic matter']
columns_VARIETIES_ONLY = np.asarray(df.iloc[:, df.columns.str.match('V\d\d\d\d\d\d')].columns)

#set the below variable to whatever columns you want to keep
columns_to_keep = columns_to_keep_top

MUST_HAVE_COLUMNS = ['Yield']
# print(columns_to_keep)

df = df.drop(columns_to_drop, axis=1) if should_drop else df
df = df.loc[:, np.concatenate((columns_to_keep, MUST_HAVE_COLUMNS))] if should_keep else df
df['YieldBucket'] = pd.Series(pd.qcut(df.Yield, q=3, labels=["high", "medium", "low"]))
print("The final dataframe has columns: ", df.columns)

The final dataframe has columns:  Index(['Yield', 'Year', 'Temperature', 'Precipitation', 'Solar Radiation',
       'Soil class', 'CEC', 'Organic matter', 'pH', 'Clay', 'Silt', 'Sand',
       'Area', 'May', 'June', 'Loc Clust 0', 'Loc Clust 1', 'Loc Clust 2',
       'Loc Clust 3', 'YieldBucket'],
      dtype='object')


In [ ]:
# LET US ALSO MAKE SURE THERE ARE NO NAN IN THE DATA
print("We expect to be %s nan values and there actually are %s nan values\n" % (0, np.sum(df.isnull().sum())))
print(df.isnull().sum())
# AFTER COLUMNS, MAKE SURE NO SKETCHY ONES
for col in df.columns:
    print(col, type(df[col][0]))    

In [8]:
# TRAIN AND TEST SPLIT# TRAIN AND TEST SPLIT# TRAIN AND TEST SPLIT
# TRAIN AND TEST SPLIT# TRAIN AND TEST SPLIT# TRAIN AND TEST SPLIT
# TRAIN AND TEST SPLIT# TRAIN AND TEST SPLIT# TRAIN AND TEST SPLIT
# TRAIN AND TEST SPLIT# TRAIN AND TEST SPLIT# TRAIN AND TEST SPLIT

X = df.drop(['Yield', 'YieldBucket'], axis=1)

print(X.columns)

y = df.Yield

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.05, train_size = 0.1, random_state = 42)

INPUT_COLS = X_train.columns
# TEST_COLS = y_train.columns

Index(['Year', 'Temperature', 'Precipitation', 'Solar Radiation', 'Soil class',
       'CEC', 'Organic matter', 'pH', 'Clay', 'Silt', 'Sand', 'Area', 'May',
       'June', 'Loc Clust 0', 'Loc Clust 1', 'Loc Clust 2', 'Loc Clust 3'],
      dtype='object')


In [9]:
print("X_train shape:", X_train.shape, "\ny_train shape:", y_train.shape)

X_train shape: (7813, 18) 
y_train shape: (7813,)


In [10]:
# This function will evaluate the errors based on RMSE (from the challenge spec)
# also will evaluate based on average error

from sklearn.metrics import mean_squared_error
def evaluate_errors(prediction, actual):
    print("RMSE Error: ", np.sqrt(mean_squared_error(prediction, actual)))
    avg_error_vector = np.absolute(((preds - y_test) / y_test) * 100)
#     print("Average Error: ", np.mean(avg_error_vector))
    print("Average Error details:\n", avg_error_vector.describe())
    return avg_error_vector

In [ ]:
from lightgbm import LGBMRegressor

clf = LGBMRegressor(n_estimators=1000, learning_rate=0.01)
clf.fit(X_train, np.log1p(y_train))
preds = np.expm1(clf.predict(X_test))
evaluate_errors(preds, y_test)

In [11]:
from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor(n_estimators=20, max_depth=13, random_state=0, verbose=1, n_jobs=-1)
regr.fit(X_train, y_train)
preds = regr.predict(X_test)

evaluate_errors(preds, y_test)

RMSE Error:  7.404785186969437
Average Error details:
 count    3907.000000
mean       10.565774
std        11.455470
min         0.002280
25%         3.511498
50%         7.616182
75%        13.677386
max       197.281042
Name: Yield, dtype: float64


[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    0.1s finished
[Parallel(n_jobs=20)]: Done   2 out of  20 | elapsed:    0.0s remaining:    0.2s
[Parallel(n_jobs=20)]: Done  20 out of  20 | elapsed:    0.0s finished


37747     3.023815
34960     7.198115
77631     2.825352
29678     3.136440
58370     3.685283
44709     6.386415
37353     2.379666
11083    11.007744
65678     9.156382
68237     7.646327
69975    12.573523
53136     4.921622
25669    29.577871
22055     6.077461
53416     2.523629
1165     20.627535
21660    27.801441
10943    17.636145
29486     3.421652
73128     3.995045
77434    10.938201
74450     7.009889
29534     5.777550
79136     4.183589
51908     5.548448
6382      7.391764
4497     11.379614
29180     8.188265
40968    14.865222
56236     5.234357
           ...    
55268     9.446306
36424     8.963997
38873     0.661564
53108     3.203873
57176     7.309249
9969      3.859484
63588     2.630800
53502    12.020692
45568     6.506807
5098     14.526325
27685    22.412765
19609    64.562177
31942    10.804728
3338     11.334763
62444    10.501975
67291     2.332348
47450     9.549224
11728     8.788222
3337     14.216159
49107    12.372128
19969    20.919254
55298    12.

In [12]:
# GET OUTPUT OF FEATURE IMPORTANCE
def get_feature_importances(regr):
    feature_importances = regr.feature_importances_
    feature_importances = pd.Series(feature_importances)
    feature_importance_df = pd.DataFrame({'feature': X_train.columns,'feature_importance': feature_importances})
    feature_importance_df = feature_importance_df.sort_values(by=['feature_importance'])
    for index, row in feature_importance_df.iterrows():
        print(row['feature'], 'has importance: ', row['feature_importance'])
get_feature_importances(regr)

Loc Clust 0 has importance:  0.012890156488064467
Loc Clust 3 has importance:  0.013915496293010268
Loc Clust 2 has importance:  0.013996485973801556
Loc Clust 1 has importance:  0.014975083610222286
Soil class has importance:  0.016145004016204077
June has importance:  0.018097253354624993
May has importance:  0.021369949647420936
Clay has importance:  0.03787393818991075
Sand has importance:  0.046254055263236775
pH has importance:  0.05433367033485603
CEC has importance:  0.055163539144148774
Area has importance:  0.06412638830101888
Solar Radiation has importance:  0.08059789601140775
Temperature has importance:  0.08516317710895183
Year has importance:  0.08614454802498779
Organic matter has importance:  0.09373563257912265
Precipitation has importance:  0.11336476020901129
Silt has importance:  0.1718529654499989


In [ ]:
import numpy as np
from sklearn import linear_model
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPRegressor

from sklearn.feature_selection import RFECV
from sklearn.ensemble import RandomForestRegressor


classifiers = [
#     MLPRegressor(solver='lbfgs', alpha=1e-5,
#                      hidden_layer_sizes=(5, 2), random_state=1),
#     RandomForestRegressor(n_estimators=20, max_depth=13, random_state=0, verbose=1, n_jobs=-1),
#     linear_model.SGDRegressor(),
#     linear_model.BayesianRidge(),
#     linear_model.LassoLars(),
#     linear_model.ARDRegression(),
#     linear_model.PassiveAggressiveRegressor(),
#     linear_model.TheilSenRegressor(),
#     linear_model.LinearRegression(),
    svm.SVR(kernel="linear"),
    linear_model.ARDRegression()]

# estimator = svm.SVR(kernel="linear")
# estimator = 

# selector = RFECV(estimator, step=1, cv=5, verbose=1, n_jobs=-1)
# selector = selector.fit(X_train, y_train)
# selector.support_ 
# # array([ True,  True,  True,  True,  True,
# #         False, False, False, False, False], dtype=bool)
# selector.ranking_
# # array([1, 1, 1, 1, 1, 6, 4, 3, 2, 5])


#     print(np.sum(preds - y_test))
#     print(clf.predict(X_test),'\n')
#     print(y_test)
#     print('accuracy score:', accuracy_score(y_test, clf.predict(X_test)), '\n')

In [ ]:
for item in classifiers:
    print(item)
    clf = item
    selector = selector = RFECV(clf, step=1, verbose=1, n_jobs=-1)
    selector = selector.fit(X_train, y_train)
    print(selector.support_)
    print(selector.ranking_)

In [ ]:
INPUT_COLS = X_train.columns

outputs = "1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1"
[ 4 10  9 11  8  7  3  2  1  1  1  1  1  5  6  1]
import re
outputs = re.sub(r'\n', '', outputs)
outputs = re.sub(r'  ', ' ', outputs)
outputs = re.sub(r' ', ',', outputs)
# print(outputs)

# outputs.replace('\n', "")
# outputs.replace("  ", ",")
output = np.asarray(outputs.split(','))
print(output)
output = output == "1"

for i in range(0, len(output)):
    if output[i] == True:
        print("CHOSEN: ", INPUT_COLS[i])
    else:
        print("NOT CHOSEN: ,", INPUT_COLS[i])
        
output

In [ ]:
from sklearn import ensemble
params = {'n_estimators': 500, 'max_depth': 4, 'min_samples_split': 2,
          'learning_rate': 0.01, 'loss': 'ls'}
clf = ensemble.GradientBoostingRegressor(**params)

clf.fit(X_train, y_train)


In [ ]:
preds = clf.predict(X_test)
errors = evaluate_errors(preds, y_test)

In [ ]:
for item in classifiers:
    print(item)
    clf = item
    clf.fit(X_train, y_train)
    preds = clf.predict(X_test)
    errors = evaluate_errors(preds, y_test)
    try:
        get_feature_importances(clf)
    except:
        print("NO FEATURE IMPORTANCE METRIC")

In [ ]:
# THIS WILL ONLY WORK WITH THE BUCKET METHOD

from sklearn.ensemble import RandomForestClassifier
regr = RandomForestClassifier(n_estimators=10, max_depth=20, random_state=0, verbose=1)
regr.fit(X_train, y_train)
preds = regr.predict(X_test)

from sklearn.metrics import accuracy_score

print(accuracy_score(y_test, preds))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

classifiers = [
#     KNeighborsClassifier(3),
#     SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
#     GaussianProcessClassifier(1.0 * RBF(1.0)),
#     DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()]
from sklearn.metrics import accuracy_score
for item in classifiers:
    print(item)
    clf = item
    clf.fit(scale(X_train), y_train)
    preds = clf.predict(scale(X_test))
    print(accuracy_score(y_test, preds))
#     errors = np.absolute(((preds - y_test) / y_test) * 100)
#     print(errors)
#     print(np.mean(errors))

In [ ]:
import random

NUM_VARIETIES = 174

def best_yield_variety(regr, test_set, random_sel = True, n_samples = 174, print_variety_preds = True):
    
    #create empty df
    dup_df = pd.DataFrame()
    
    #choose a rand sample of input test_set (for dev purposes, wouldn't be used in app)
    test_set_sample = test_set.sample(n= n_samples) if random_sel else test_set
    
    #progress of intensive, long for loop upcoming
    counter = 0
    
    #for loop will, for each row in test_set_sample, duplicate that row by NUM_VARIETIES
    for index, row in test_set_sample.iterrows():
        counter+=1
        print(counter)
        dup_df = dup_df.append([row] * NUM_VARIETIES, ignore_index = True)
        
    #extract the varieties columns
    duplicated_df_varieties = dup_df.loc[:, dup_df.columns.str.match('V\d\d\d\d\d\d')]
    #extract the names of the varieties
    varieties_array = duplicated_df_varieties.columns
    num_expanded_data_pts = duplicated_df_varieties.shape[0]
    #we must have a zeroed matrix of the same shape as the duplicated_df_varieties
    #so that we can input a 1 just once for each variety per 174 rows
    d = np.zeros((duplicated_df_varieties.shape[0], NUM_VARIETIES))
    #make d our dataframe, with columns equal to the varieties
    duplicated_df_varieties = pd.DataFrame(d, columns=varieties_array)
    #for loop will place a 1 just once for each variety per 174 rows (one hot rep)
    for i in range(duplicated_df_varieties.shape[0]):
        var_index = i % 174
        duplicated_df_varieties.loc[i, varieties_array[var_index]] = 1
    #remove the varieties (will be added back with the new values)
    dup_df = dup_df.drop(varieties_array, axis = 1)
    #add the new values (one hot representations) from above for loop
    dup_df = pd.concat([dup_df, duplicated_df_varieties], axis=1)
    #do prediction on the entire dataframe
    preds_per_variety = regr.predict(dup_df)
    #*******make it into a dataframe where each row will give the performance of each variety
    #with the same environmental conditions*******
    preds_df = pd.DataFrame(preds_per_variety.reshape((int(num_expanded_data_pts/NUM_VARIETIES), NUM_VARIETIES)),
                            columns=varieties_array)
    
    #environmental conditions (everything except the variety data)
    envcond = test_set_sample.drop(varieties_array, axis=1)
    
    #a simple print out for best variety given the environmental conditions
    hr_preds = []
    if print_variety_preds:
        envcond_cols = envcond.columns
        counter = -1
        for idx, row in envcond.iterrows():
            counter+=1
            out = "For environmental conditions:\n%s\nthe best variety is:%s" % (row, preds_df.idxmax(axis=1)[counter])
            hr_preds.append(out)
            print(out)
    
    return preds_df, envcond, hr_preds
    
        
preds_df, envcond, hr_preds = best_yield_variety(regr, X_test, n_samples = 500)

In [ ]:
#find number of times a variety is the maximum in the above prediction dataframe
maxes = preds_df.idxmax(axis=1) # get first max variety per env
# print(pd.get_dummies(maxes).describe())
print(pd.get_dummies(maxes).sum().sort_values(ascending=False))

In [ ]:
#find number of times a variety is the maximum in the above prediction dataframe
maxes = preds_df.idxmax(axis=1) # get first max variety per env
# print(pd.get_dummies(maxes).describe())
print(pd.get_dummies(maxes).sum().sort_values(ascending=False))
print(len(pd.get_dummies(maxes).sum()))

In [ ]:
print(hr_preds[0])

In [ ]:
preds_df.describe().sort_values(by="mean", axis=1)

In [ ]:
preds_df.describe().loc['mean',:].sort_values().describe()